# Text Rank

In [2]:
import requests
from bs4 import BeautifulSoup
import re


def get_news_by_url(url):
    headers = {"user-agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = soup.select_one(
        "#articleBodyContents").get_text().replace("\n", "")
    content = content.replace(
        "// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")

    start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
    content = content[:start_pos-1]
    return content


doc = get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc = re.sub("[^가-힣 \d]", " ", doc)
doc

'과기정통부  22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입  5100여종 데이터 구축민간 클라우드 통한 외부연계체계도   개방성 강화  이데일리 이재운 기자  국가 차원의 빅데이터 활용 시대가 열린다  새로운 산업 창출과 기존 산업의 변화에 이르는  혁신성장 을 위한 센터가 문을 연다  10개 분야에 걸쳐  데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한  빅데이터 플랫폼 및 센터  출범식 행사를 개최했다  유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다  10개 분야 100개 센터  3년간 1516억원 투입이미지  픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다  문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고  플랫폼은 이를 수집 분석 유통하는 역할을 담당한다  과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며  올해 우선 640억원 규모의 사업을 추진하고 있다 대상 분야는  금융   카드   환경 한국수자원공사   문화 한국문화정보원   교통 한국교통연구원   헬스케어 국립암센터   유통 소비 매일방송   통신      중소기업 더존비즈온   지역경제 경기도청   산림 한국임업진흥원  등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고  다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원  운영할 계획이다  이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다 주요 활용 전략 사례를 보면 빅데이터 활용을 

## Tokenize words using Mecab

In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

# tokens = doc 에서 POS tagging 한것
tokens = [token for token in mecab.pos(doc)]

#nodes = 모든 token 중 POS tag 가 없는것
nodes = [t[0] for t in tokens]

#vocab = 불용어 처리한것
vocab = [t[0] for t in tokens if t[1] in ['NNG', 'NNP'] and len(t[0]) > 1]

print(doc[:10])
print(tokens[:10])
print(nodes[:10])
print(vocab[:10])

과기정통부  22일
[('과기', 'NNG'), ('정통부', 'NNG'), ('22', 'SN'), ('일', 'NNBC'), ('유영민', 'NNP'), ('장관', 'NNG'), ('등', 'NNB'), ('참석', 'NNG'), ('해', 'XSV+EC'), ('기념행사', 'NNG')]
['과기', '정통부', '22', '일', '유영민', '장관', '등', '참석', '해', '기념행사']
['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '데이터', '구축']


## Give a unique ID to vocab

In [5]:
vocab = list(set(vocab))

vocab2idx = {vocab[i]: i for i in range(len(vocab))}
idx2vocab = {i: vocab[i] for i in range(len(vocab))}

In [6]:
import numpy as np
import math

vocab_len = len(vocab2idx)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len), dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 3
covered_coocurrences = []

for window_start in range(len(nodes) - window_size + 1):
    window = nodes[window_start:window_start+window_size]

    for i in range(window_size):
        for j in range(i+1, window_size):
            if window[i] in vocab and window[j] in vocab:
                index_i = window_start + i
                index_j = window_start + j

                if (index_i, index_j) not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]
                                  ][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]
                                  ][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append((index_i, index_j))

for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001  # convergence threshold

for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            if weighted_edge[j][i] != 0:
                summation += weighted_edge[j][i] * prev_score[j]

        score[i] = (1 - d) * d*summation

    if np.sum(np.fabs(prev_score - score)) <= threshold:
        break


sorted_index = np.flip(np.argsort(score), 0)

n = 5


print("\n=== 핵심키워드 ===")
for i in range(0, n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
데이터 : 7.405189e-07
센터 : 2.49831e-07
한국 : 2.0378042e-07
활용 : 1.9834519e-07
대한 : 1.9389464e-07


# Mini note

![ex_screenshot](../img/textrank1.png)

In [8]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

# [vocab2idx[token] for token in vocab] 

vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

![ex_screenshot](../img/textrank2.png)

In [17]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_cooccurence = []

tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']

for window_start in range(0, (len(tokens) - window_size + 1)):
    window = tokens[window_start : window_start + window_size]
    print("window_start",window_start)
    print(window)
    
    for i in range(window_size):
        print("i",i)
        for j in range(i + 1, window_size):
            print("j",j)
            if(window[i] in vocab and window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                print(vocab2idx[window[i]])
                print(vocab2idx[window[j]])


                if (index_i, index_j) not in covered_cooccurence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))


for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    print(f"{i} : {row_sum}")
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

print(weighted_edge)

window_start 0
['딸기', '바나나']
i 0
j 1
3
0
i 1
window_start 1
['바나나', '사과']
i 0
j 1
0
1
i 1
window_start 2
['사과', '딸기']
i 0
j 1
1
3
i 1
window_start 3
['딸기', '파인애플']
i 0
j 1
3
2
i 1
0 : 2.0
1 : 2.0
2 : 1.0
3 : 3.0
[[0.         0.5        0.         0.5       ]
 [0.5        0.         0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333334 0.33333334 0.33333334 0.        ]]


In [18]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(MAX_ITERATIONS):
    
    # threshold 때문에 만듬
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * prev_score[j]

        score[i] = (1-d) + d*summation
        print("***** score *****")
        print(score)
    
    # np.fabs flaot 절대값(abs)
    if np.sum(np.fabs(prev_score - score)) <= threshold:
        break

print("***** final *****")
print(score)


***** score *****
[0.85833335 1.         1.         1.        ]
***** score *****
[0.85833335 0.85833335 1.         1.        ]
***** score *****
[0.85833335 0.85833335 0.43333334 1.        ]
***** score *****
[0.85833335 0.85833335 0.43333334 1.85      ]
***** score *****
[1.0389583  0.85833335 0.43333334 1.85      ]
***** score *****
[1.0389583  1.0389583  0.43333334 1.85      ]
***** score *****
[1.0389583 1.0389583 0.6741667 1.85     ]
***** score *****
[1.0389583 1.0389583 0.6741667 1.2479167]
***** score *****
[0.9451337 1.0389583 0.6741667 1.2479167]
***** score *****
[0.9451337 0.9451337 0.6741667 1.2479167]
***** score *****
[0.9451337 0.9451337 0.5035764 1.2479167]
***** score *****
[0.9451337 0.9451337 0.5035764 1.6061562]
***** score *****
[1.0067594 0.9451337 0.5035764 1.6061562]
***** score *****
[1.0067594 1.0067594 0.5035764 1.6061562]
***** score *****
[1.0067594 1.0067594 0.6050776 1.6061562]
***** score *****
[1.0067594 1.0067594 0.6050776 1.3814036]
***** score ****

In [19]:
# np.argsort() <-- sort array and return index
sorted_index = np.flip(np.argsort(score),0) 

n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4669281
사과 : 0.9837155
바나나 : 0.9837155
파인애플 : 0.56564105
